In [ ]:
#!pip install oracledb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00


In [5]:
def digita_inteiro(texto):
  while True:
    try:
      inteiro = int(input(texto).strip())
      return inteiro
    except ValueError:
      print("Entrada inválida. Por favor, digite um número inteiro.")

In [17]:
import json
import pandas as pd
import oracledb
from pathlib import Path
import os

def start_conn():
    try:
        DATA_FILE = "oracle_conn.json"
        file = open(DATA_FILE, 'r')
        xfile = json.load(file)

        connection = oracledb.connect(
            user=xfile[0]["user"],
            password=xfile[0]["password"],
            dsn=xfile[0]["dsn"]
        )

        cursor = connection.cursor()
        print("Conexão bem-sucedida!")
        return connection, cursor

    except FileNotFoundError:
        print('Arquivo não encontrado. Verifique o caminho.')
        return None, None

    except json.JSONDecodeError:
        print('Erro ao decodificar o arquivo JSON. Verifique a sintaxe.')
        return None, None

    except KeyError as e:
        print(f'Chave ausente no JSON: {e}')
        return None, None

    except oracledb.DatabaseError as e:
        error, = e.args
        print("Erro de banco de dados:", error.message)
        return None, None

    except Exception as e:
        print("Erro inesperado:", e)
        return None, None

In [18]:
connection, cursor = start_conn()

Conexão bem-sucedida!


In [19]:
def insert_record():

    nome = input("Digite o nome: ").strip()
    sobrenome = input("Digite o sobrenome: ").strip()
    telefone = digita_inteiro("Digite o telefone: ")

    try:
        # Verifica se a tabela "AGENDA" já existe
        check_table_query = """
        SELECT table_name FROM user_tables WHERE table_name = 'AGENDA'
        """
        cursor.execute(check_table_query)
        result = cursor.fetchone()

        if result:
            print("A tabela 'AGENDA' já existe.")
        else:
            # Criação da tabela
            create_table_query = '''
            CREATE TABLE AGENDA (
                id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                nome VARCHAR2(50),
                sobrenome VARCHAR2(50),
                telefone VARCHAR2(50)
            )
            '''
            cursor.execute(create_table_query)
            print("Tabela 'AGENDA' criada com sucesso.")

    except oracledb.DatabaseError as e:
        error, = e.args
        print("Erro de banco de dados:", error.message)

    except Exception as e:
        print("Erro inesperado:", e)

    try:
        # Inserção de um novo registro
        insert_query = '''
        INSERT INTO AGENDA (nome, sobrenome, telefone)
        VALUES (:nome, :sobrenome, :telefone)
        '''

        cursor.execute(insert_query, nome=nome, sobrenome=sobrenome, telefone=telefone)
        connection.commit()
        print("Registro inserido com sucesso!")

    except oracledb.DatabaseError as e:
        error, = e.args
        print("Erro de banco de dados:", error.message)

    except Exception as e:
        print("Erro inesperado:", e)

In [34]:
insert_record()

Digite o nome: Joao
Digite o sobrenome: Silva
Digite o telefone: 345345453
A tabela 'AGENDA' já existe.
Registro inserido com sucesso!


In [32]:
def list_records():
    try:
        cursor.execute("SELECT * FROM AGENDA")

        # Buscando e imprimindo todas as tabelas
        agenda_records = cursor.fetchall()

    except oracledb.DatabaseError as e:
        error, = e.args
        print("Erro de banco de dados:", error.message)

    except Exception as e:
        print("Erro inesperado:", e)

    print("Lista de registros:\n")
    print(f"{'Nome':<10} {'Sobrenome':<20} - Telefone")
    for registro in agenda_records:
        print(f"{registro[1]:<10} {registro[2]:<20} - {registro[3]:<15}")

In [36]:
list_records()

Lista de registros:

Nome       Sobrenome            - Telefone
Luiz       tavares              - 123123123123   
Joao       Silva                - 345345453      


In [48]:
def search_name(nome_busca):
    encontrado = False
    agenda_records = []

    try:
        query = f'''SELECT * FROM AGENDA WHERE nome = '{nome_busca}' '''

        cursor.execute(query)

        # Buscando e imprimindo todas as tabelas
        agenda_records = cursor.fetchall()

    except oracledb.DatabaseError as e:
        error, = e.args
        print("Erro de banco de dados:", error.message)

    except Exception as e:
        print("Erro inesperado:", e)

    for linha in agenda_records:
        if linha[1].strip().lower() == nome_busca.strip().lower():
            print(f"{'Nome':<10} {'Sobrenome':<20} - Telefone")
            print(f"{linha[1]:<10} {linha[2]:<20} - {linha[3]:<15}")
            encontrado = True

    if not encontrado:
        print("Nenhum registro encontrado com esse nome.")

In [51]:
search_name("Luiz")

Nome       Sobrenome            - Telefone
Luiz       tavares              - 123123123123   


In [ ]:
def delete_record(nome_remover):

    print(f"Registros com nome '{nome_remover}' removidos com sucesso.")

In [ ]:
delete_record("Alberto")

Registros com nome 'Alberto' removidos com sucesso.


In [ ]:
def update_record(nome_busca):
    atualizados = 0

    print(f"\n{atualizados} registro(s) atualizado(s).")
    return atualizados

In [ ]:
update_record("Jorge")


Registro atual: Jorge Campos - 234242
Novo nome (Enter para manter): 
Novo sobrenome (Enter para manter): 
Novo Telefone (Enter para manter): 

1 registro(s) atualizado(s).


1

In [ ]:
def menu():
    print("\n=== Lista Telefônica ===")
    print("1) Criar registro")
    print("2) Listar registros")
    print("3) Atualizar registro (por nome)")
    print("4) Buscar registro (por nome)")
    print("5) Remover registro (por nome)")
    print("0) Sair")

if __name__ == "__main__":
    start_conn()

    while True:
        menu()
        opcao = input("Escolha uma opção: ").strip()

        if opcao == "1":
            insert_record()

        elif opcao == "2":
            print("\n")
            list_records()

        elif opcao == "3":
            nome = input("Nome a atualizar: ").strip()
            if nome:
                update_record(nome)
                print("\n")
                list_records()
            else:
                print("Informe um nome válido.")

        elif opcao == "4":
            nome = input("Nome para busca: ").strip()
            if nome:
                print("\n")
                search_name(nome)
            else:
                print("Informe um nome válido.")

        elif opcao == "5":
            nome = input("Nome para remover: ").strip()
            if nome:
                print("\n")
                delete_record(nome)
                print("\n")
                list_records()
            else:
                print("Informe um nome válido.")

        elif opcao == "0":
            print("Saindo.")
            break

        else:
            print("Opção inválida. Tente novamente.")


=== Lista Telefônica ===
1) Criar registro
2) Listar registros
3) Atualizar registro (por nome)
4) Buscar registro (por nome)
5) Remover registro (por nome)
0) Sair
Escolha uma opção: 2


Lista de registros:

Nome       Sobrenome            -        Telefone
Jorge      Campos               -          234242
Luiz       Tavares              -       345345345
Otavio     Neves                -         4444444
qweqwe     qweqwe               -          121231

=== Lista Telefônica ===
1) Criar registro
2) Listar registros
3) Atualizar registro (por nome)
4) Buscar registro (por nome)
5) Remover registro (por nome)
0) Sair
Escolha uma opção: 5
Nome para remover: qweqwe


Registros com nome 'qweqwe' removidos com sucesso.


Lista de registros:

Nome       Sobrenome            -        Telefone
Jorge      Campos               -          234242
Luiz       Tavares              -       345345345
Otavio     Neves                -         4444444

=== Lista Telefônica ===
1) Criar registro
2) List

# **Exemplo de funcao para digitacao**

In [ ]:
def digita_inteiro(texto):
  while True:
    try:
      inteiro = int(input(texto).strip())
      return inteiro
    except ValueError:
      print("Entrada inválida. Por favor, digite um número inteiro.")

In [ ]:
telefone = digita_inteiro("Digite o telefone: ")

Digite o telefone: 1231231


In [ ]:
telefone

1231231